In [2]:
import pandas as pd
import numpy as np

In [3]:
import random
from Functions import  xyzrange, eigen, mean, normalise, Min, Max, best_fitting_plane, eig, std_dev, PCA

In [4]:
from sklearn.neighbors import KDTree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn import preprocessing

In [5]:
df = pd.read_csv('CentralStation_BuildingTree.csv')

all_rows = df.iloc[0:len(df)][['X','Y','Z']]
""" Create matrix of data """
Lt = np.array(all_rows)

kdt = KDTree(all_rows, leaf_size=50, metric='euclidean') # data structure for nearest neighbour

F = df.iloc[0:len(df)][['X','Y','Z','R','G','B','Intensity']]
Ft = np.array(F)

#feature lists

f3 = []
ev = []
normies = []


"""Calculate attribute data to feed classifier """
for i in range(len(df)):

    dist, ind = kdt.query([Lt[i]], k=50)
    x = []
    y = []
    z = []
    Int = []
    R = []
    G = []
    B = []
    for k in range(len(ind[0])):
        indice = ind[0][k]
        
        x.append(Ft[indice][0]) #'''list of xyz -cords of indices'''
        y.append(Ft[indice][1])
        z.append(Ft[indice][2])
        Int.append(Ft[indice][6])
        R.append(Ft[indice][3])
        G.append(Ft[indice][4])
        B.append(Ft[indice][5])
    
    R_normalized = preprocessing.normalize([R], norm='l2') #normalisation of rgb, intensities
    G_normalized = preprocessing.normalize([G], norm='l2')
    B_normalized = preprocessing.normalize([B], norm='l2')
    Int_normalized = preprocessing.normalize([Int], norm='l2')
    Int_std = std_dev(Int_normalized.tolist()[0]) #standard deviation of intensities
    data = np.c_[x,y,z]
    pt, normal = best_fitting_plane(data)
    eigenv_x = PCA(data)[0][0]    
    eigenv_y = PCA(data)[0][1]    
    eigenv_z = PCA(data)[0][2]
    ev.append([eigenv_x,eigenv_y,eigenv_z])

ex = preprocessing.normalize([np.array(ev)[:,0].tolist()]) #normalisation of eigenvalues
ey = preprocessing.normalize([np.array(ev)[:,1].tolist()])
ez = preprocessing.normalize([np.array(ev)[:,2].tolist()])   


for i in range(len(df)):
    f3.append([ex[0][i],ey[0][i],ez[0][i]])

In [6]:
kmeans3 = KMeans(n_clusters=6, random_state=0).fit(f3)
kmeans_labels3 = kmeans3.labels_
labels3 = {'X': Lt[:,0], 'Y': Lt[:,1], 'Z': Lt[:,2], 'kmeans labels': kmeans_labels3}
labels_new3 = pd.DataFrame(data = labels3)
labels_new3.to_csv('citycenter.csv',index=None)